In [5]:
from tensorflow.keras import mixed_precision
import tensorflow as tf

# Load the model with correct policy
mixed_precision.set_global_policy("mixed_float16")
model = tf.keras.models.load_model("EyeNet_Model.h5")

# Convert all layers to float32
for layer in model.layers:
    if hasattr(layer, 'dtype') and layer.dtype == 'float16':
        layer._dtype_policy = mixed_precision.Policy('float32')

# Rebuild model to enforce float32 everywhere
config = model.get_config()
weights = model.get_weights()
with tf.keras.utils.custom_object_scope({'SigmoidFocalCrossEntropy': tf.keras.losses.BinaryCrossentropy}):
    model_float32 = tf.keras.Model.from_config(config)
model_float32.set_weights(weights)

# Save the pure float32 model
model_float32.save("EyeNet_Model_float32.h5")
print("Saved pure float32 model.")

TypeError: Exception encountered when calling layer "tf.compat.v1.nn.depthwise_conv2d_1" (type TFOpLambda).

Input 'filter' of 'DepthwiseConv2dNative' Op has type float32 that does not match type float16 of argument 'input'.

Call arguments received by layer "tf.compat.v1.nn.depthwise_conv2d_1" (type TFOpLambda):
  • input=tf.Tensor(shape=(None, 12, 12, 1024), dtype=float16)
  • filter=tf.Tensor(shape=(3, 3, 1024, 2), dtype=float32)
  • strides=['1', '1', '1', '1']
  • padding='VALID'
  • rate=None
  • name=None
  • data_format=None
  • dilations=None

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from tensorflow.keras import mixed_precision


mixed_precision.set_global_policy('float32')

model_path = "EyeNet_Model.h5"
test_dir = r"C:\Users\User\Desktop\dataset_split\test"

# Do NOT set mixed_float16 here
model = load_model(model_path)

datagen = ImageDataGenerator(rescale=1.0 / 255)
test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=(320, 320),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

# Get true & predicted labels
y_true = []
y_pred = []

for batch_x, batch_y in test_gen:
    preds = model.predict(batch_x)
    y_true.extend(np.argmax(batch_y, axis=1))
    y_pred.extend(np.argmax(preds, axis=1))
    if len(y_true) >= 1000:  # stop after full test set
        break

y_true = np.array(y_true[:1000])
y_pred = np.array(y_pred[:1000])

# Confusion matrix & report
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

report = classification_report(
    y_true, y_pred,
    target_names=["Cataract", "Conjunctivitis", "Normal", "Stye"]
)
print(report)


TypeError: Exception encountered when calling layer "tf.compat.v1.nn.depthwise_conv2d_1" (type TFOpLambda).

Input 'filter' of 'DepthwiseConv2dNative' Op has type float32 that does not match type float16 of argument 'input'.

Call arguments received by layer "tf.compat.v1.nn.depthwise_conv2d_1" (type TFOpLambda):
  • input=tf.Tensor(shape=(None, 12, 12, 1024), dtype=float16)
  • filter=tf.Tensor(shape=(3, 3, 1024, 2), dtype=float32)
  • strides=['1', '1', '1', '1']
  • padding='VALID'
  • rate=None
  • name=None
  • data_format=None
  • dilations=None